In [1]:
from iuvs import io, scaling

In [2]:
summaryfile = io.l1b_filenames('summary', ext='.txt')[0]

In [3]:
fname = io.l1b_filenames("cruisecal2-mode080-muv", stage=False)

In [4]:
l1b = io.L1BReader(fname)

In [5]:
dark0, dark1, dark2 = l1b.detector_dark

(62, 341)